In [9]:
import numpy as np
import cv2
import math

In [23]:
feature_params = dict(maxCorners=10,qualityLevel=0.5,minDistance=7,blockSize=7)
lk_params= dict(winSize=(15,15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

font=cv2.FONT_HERSHEY_SIMPLEX


In [29]:
cap=cv2.VideoCapture('basketball.mp4')
ret,prevFrame=cap.read()
prev_gray=cv2.cvtColor(prevFrame,cv2.COLOR_BGR2GRAY)
prevPts=cv2.goodFeaturesToTrack(prev_gray,mask=None,**feature_params)
mask=np.zeros_like(prevFrame)
counter=0
speed=0

fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1 / fps

while True:
    ret,frame=cap.read()

    if not ret:
        break
    frame_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    newPts,status,err=cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)

    if newPts is not None:
        good_new=newPts[status==1]
        good_old=prevPts[status==1]

        for new,old in zip(good_new,good_old):
            x_new,y_new=new.ravel()
            x_old,y_old=old.ravel()

            distance=math.sqrt(abs(x_new-x_old)**2+abs(y_new-y_old)**2)

            if distance >= 20:
                counter+=1
                speed=distance/frame_time
                mask = np.zeros_like(frame)  

            mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)
        frame=cv2.add(frame,mask)
        cv2.putText(frame, f'Count: {counter}', (10, 50), font, 1, (0, 255, 255), 2)
        cv2.putText(frame, f'speed: {speed}', (10, 80), font, 1, (0, 255, 255), 1) #px/second


        cv2.imshow("Tracker", frame)

        key = cv2.waitKey(20) & 0xFF
        if key == 27:
            break

        prev_gray=frame_gray
        prevPts = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()